<a href="https://colab.research.google.com/github/vtsimoes/class_victor_dataset/blob/main/G_Pre_process_VICTOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pré-processamento dos dados do DATASET VICTOR


In [ ]:
#Load required libraries
import os
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib import pylab
import time
import pickle
from google.colab import drive
import string
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
import random

Funções para salvar os dados pré-processados

In [ ]:
def save_pickle(data_to_save,path='',filename='file.pkl'):
  open_file = open(path + filename, "wb")
  pickle.dump(data_to_save, open_file)
  open_file.close()

def load_pickle(path='',filename='file.pkl'):
  open_file = open(path + filename, "rb")
  pkl_file = pickle.load(open_file)
  open_file.close()
  return pkl_file

In [ ]:
#definição dos paths com os Dados do dataset
path = ''
path_dados = ''
path_preprocessados = ''
#Montando o google drive para obter os dados
drive.mount('/content/drive/')

Mounted at /content/drive/


Carregamento do DATASET VICTOR

In [ ]:
filename_test = "test_small.csv"
filename_train = "train_small.csv"
filename_validation = "validation_small.csv"

In [ ]:
train = pd.read_csv(path+'small/'+filename_train)
valid = pd.read_csv(path+'small/'+ filename_validation)
test = pd.read_csv(path+'small/'+filename_test)

### Testando o carregamento dos dados e entendendo o formato

In [ ]:
test.loc[test.body.str.contains('LEI_')]

In [ ]:
test.loc[(test.file_name == 'AI_859014_2811603_50_16052013.pdf') & (test.pages == 3)].body.values

array(['{"colendo supremo tribunal federal senhores ministros razões recursais síntese decisão recorrida trata de decisão presidente turma recursal dos juizados especiais seção judiciária estado paraíba que negou seguimento recurso extraordinário união nos seguintes termos decisão cuida de recurso extraordinário interposto contra acórdão deste órgão colegiado que rejeitou alegação prescrição negou seguimento recurso ente público mantendo sentença recorrida por seus próprios fundamentos eis que calçada posicionamento reiterado supremo tribunal federal dispõe LEI_10259 que instituiu juizados especiais federais cíveis criminais âmbito justiça federal ARTIGO_14 caberá pedido uniformização interpretação lei federal quando houver divergência entre decisões sobre questões direito material proferidas por turmas recursais interpretação LEI_4º quando orientação acolhida pela turma uniformização questões direito material contrariar súmula jurisprudência dominante superior tribunal justiça stj par

## Pré-processamento para remoção de vazios, e tokens desnecessários

###Removendo "{}" do campo de textos

In [ ]:
def strip_trash(df, column="body"):
    df[column] = df[column].str.strip('{}"')
    return df

In [ ]:
train_clean = strip_trash(train)
valid_clean = strip_trash(valid)
test_clean = strip_trash(test)

In [ ]:
del(train)
del(valid)
del(test)

In [ ]:
!python -m spacy download pt_core_news_md #Carregando o módulo de português do Brasil do Spacy

In [ ]:
import spacy
nlp = spacy.load("pt_core_news_md")

In [ ]:
def preProcessData(df,remove_stopwords=True,lemma = False):
  docs = []
  labels = []
  file_name = []
  pages = []
  trunked = []
  stops = ['fls.','fls','fl.','fl','pg.','pg']
  if remove_stopwords:
    stop_words = set(stopwords.words('portuguese'))
    for stop in stops:
      stop_words.add(stop)
  else:
    print('Sem remoção de stopwords')
    stop_words = set(stops)
  #padrão para remoção de links
  for i, row in df.iterrows():
    labels.append(row.document_type)
    file_name.append(row.file_name)
    pages.append(row.pages)
    doc = re.sub(r"http\S+", "", row.body) #remove links
    doc = doc.lower()
    if lemma:
      lemmatize = nlp(doc)
      tokens = []
      for token in lemmatize:
        tokens.append(token)
      doc = ' '.join([token.lemma_ for token in tokens])

    docs.append(doc)

  se_doc = pd.Series(data=docs)
  df_doc = pd.DataFrame(data=se_doc,columns=['body'])
  df_doc['document_type'] = labels
  df_doc['file_name'] = file_name
  df_doc['pages'] = pages
  df_doc['body'] = df_doc['body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
  return df_doc

In [ ]:
df_train_processed = preProcessData(train_clean,remove_stopwords=True, lemma= True)
del(train_clean)
df_test_processed = preProcessData(test_clean,remove_stopwords=True, lemma= True)
del(test_clean)
df_valid_processed = preProcessData(valid_clean,remove_stopwords=True, lemma= True)
del(valid_clean)

In [ ]:
# Função para exibir as quantidade de documentos
def quantidades(df):
  print(df.document_type.value_counts(), df.document_type.value_counts()/len(df))

###Removendo dados vazios

In [ ]:
df_train_proc_sem_vazio =  df_train_processed.loc[df_train_processed.body.str.len() != 0]
del(df_train_processed)
df_test_proc_sem_vazio =  df_test_processed.loc[df_test_processed.body.str.len() != 0]
del(df_test_processed)
df_valid_proc_sem_vazio =  df_valid_processed.loc[df_valid_processed.body.str.len() != 0]
del(df_valid_processed)

In [ ]:
quantidades(df_train_proc_sem_vazio)

In [ ]:
quantidades(df_test_proc_sem_vazio)

In [ ]:
quantidades(df_valid_proc_sem_vazio)

Salvando os dados pré-processados

In [ ]:
save_pickle(df_train_proc_sem_vazio,path=path_dados+path_preprocessados,filename='train_processed_per_page_sw_lemma.pkl')
save_pickle(df_valid_proc_sem_vazio,path=path_dados+path_preprocessados,filename='valid_processed_per_page_sw_lemma.pkl')
save_pickle(df_test_proc_sem_vazio,path=path_dados+path_preprocessados,filename='test_processed_per_page_sw_lemma.pkl')

Carregando os documentos pré-processados e sem as páginas unidas para teste de pré-processamento

In [ ]:
test = load_pickle(path=path_dados+path_preprocessados,filename='test_processed_per_page_sw_lemma.pkl')

In [ ]:
test.loc[(test.file_name == 'AI_859014_2811603_50_16052013.pdf') & (test.pages == 3)].body.values

array(['coler Supremo tribunal federal senhor ministro razão recursal síntese decisão recorrir tratar decisão presidente turma recursal juizado especial seção judiciário estado paraíba negar seguimento recurso extraordinário união seguinte termo decisão cuidar recurso extraordinário interposto contra acórdão órgão colegiado rejeitar alegação prescrição negar seguimento recurso ente público manter sentença recorrir próprio fundamento eis calçada posicionamento reiterar Supremo tribunal federal dispor LEI_10259 instituir juizado especial federal cível criminal âmbitor justiça federal ARTIGO_14 caber pedir uniformização interpretação lei federal divergência decisão sobre questão direito material proferir turma recursal interpretação LEI_4º orientação acolhir turma uniformização questão direito material contrariar súmula jurisprudência dominante alto tribunal Justiça stj parte interessado poder provocar manifestação dirimir divergência caso presente plausibilidade direito invocar fundar re



*  Texto Original



"colendo supremo tribunal federal senhores ministros razões recursais síntese decisão recorrida trata de decisão presidente turma recursal dos juizados especiais seção judiciária estado paraíba que negou seguimento recurso extraordinário união nos seguintes termos decisão cuida de recurso extraordinário interposto contra acórdão deste órgão colegiado que rejeitou alegação prescrição negou seguimento recurso ente público mantendo sentença recorrida por seus próprios fundamentos eis que calçada posicionamento reiterado supremo tribunal federal dispõe LEI_10259 que instituiu juizados especiais federais cíveis criminais âmbito justiça federal ARTIGO_14 caberá pedido uniformização interpretação lei federal quando houver divergência entre decisões sobre questões direito material proferidas por turmas recursais interpretação LEI_4º quando orientação acolhida pela turma uniformização questões direito material contrariar súmula jurisprudência dominante superior tribunal justiça stj parte interessada poderá provocar manifestação deste que dirimirá divergência caso presente plausibilidade direito invocado havendo fundado receio dano difícil reparação poderá relator"



*   Texto Após pré-processamento

'coler Supremo tribunal federal senhor ministro razão recursal síntese decisão recorrir tratar decisão presidente turma recursal juizado especial seção judiciário estado paraíba negar seguimento recurso extraordinário união seguinte termo decisão cuidar recurso extraordinário interposto contra acórdão órgão colegiado rejeitar alegação prescrição negar seguimento recurso ente público manter sentença recorrir próprio fundamento eis calçada posicionamento reiterar Supremo tribunal federal dispor LEI_10259 instituir juizado especial federal cível criminal âmbitor justiça federal ARTIGO_14 caber pedir uniformização interpretação lei federal divergência decisão sobre questão direito material proferir turma recursal interpretação LEI_4º orientação acolhir turma uniformização questão direito material contrariar súmula jurisprudência dominante alto tribunal Justiça stj parte interessado poder provocar manifestação dirimir divergência caso presente plausibilidade direito invocar fundar receio dano difícil reparação poder relator'



In [ ]:
df_train_proc_sem_vazio = load_pickle(path=path_dados+path_preprocessados,filename='train_processed_per_page.pkl')
df_valid_proc_sem_vazio = load_pickle(path=path_dados+path_preprocessados,filename='valid_processed_per_page.pkl')
df_test_proc_sem_vazio = load_pickle(path=path_dados+path_preprocessados,filename='test_processed_per_page.pkl')

##Juntando as páginas em um único documento

Lembrando que o Dataset VICTOR separa os documentos por página em cada linha do dataframe, nesse caso, para trabalhar com o documento inteiro é necessário juntar essas páginas.

In [ ]:
df_train = df_train_proc_sem_vazio
del(df_train_proc_sem_vazio)
df_valid = df_valid_proc_sem_vazio
del(df_valid_proc_sem_vazio)
df_test = df_test_proc_sem_vazio
del(df_test_proc_sem_vazio)

In [ ]:
#Conjunto de treino
df_train['body'] = df_train['body'].astype('str')
documents_train = df_train.groupby(['file_name'])['body'].apply(lambda x: ' '.join(x))
doc_type = df_train.groupby(['file_name']).agg({'pages': 'last','document_type': 'last'})
documents_train = pd.concat([documents_train,doc_type],axis=1)
documents_train.reset_index(inplace=True)
del(df_train)

In [ ]:
#Conjunto de validacao
df_valid['body'] = df_valid['body'].astype('str')
documents_valid = df_valid.groupby(['file_name'])['body'].apply(lambda x: ' '.join(x))
doc_type = df_valid.groupby(['file_name']).agg({'pages': 'last','document_type': 'last'})
documents_valid = pd.concat([documents_valid,doc_type],axis=1)
documents_valid.reset_index(inplace=True)
del(df_valid)

In [ ]:
#Conjunto de testes
df_test['body'] = df_test['body'].astype('str')
documents_test = df_test.groupby(['file_name'])['body'].apply(lambda x: ' '.join(x))
doc_type = df_test.groupby(['file_name']).agg({'pages': 'last','document_type': 'last'})
documents_test = pd.concat([documents_test,doc_type],axis=1)
documents_test.reset_index(inplace=True)
del(df_test)
del(doc_type)

Mostrando as quantidades

In [ ]:
quantidades(documents_train)

In [ ]:
quantidades(documents_valid)

In [ ]:
quantidades(documents_test)

Salvando documentos com as páginas unidas

In [ ]:
save_pickle(documents_train,path=path_dados+path_preprocessados,filename='train_processed_concat_pages_sw_lemma.pkl')
save_pickle(documents_test,path=path_dados+path_preprocessados,filename='test_processed_concat_pages_sw_lemma.pkl')
save_pickle(documents_valid,path=path_dados+path_preprocessados,filename='valid_processed_concat_pages_lemma_sw.pkl')